# Support vector machines

In [1]:
# import libraries

import numpy as np
import math
import pandas as pd

from matplotlib.backends.backend_pdf import PdfPages

from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import pairwise_kernels

from sklearn import model_selection

from sklearn.impute import KNNImputer

## Data pre-processing

In [2]:
# load training data

# load data from csv file
df_train_features = pd.read_csv ('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

# Load test data
df_test_features = pd.read_csv ('test_features.csv')

 ### Histogram of the output labels 

We should check for class imbalance.

In [3]:
df_train_labels.hist()

with PdfPages("./Results/Labels_histogram.pdf") as export_pdf:
    for i in list(df_train_labels)[1:]:
        df_train_labels.hist(column = i, bins = 100)
        export_pdf.savefig()

Error in callback <function flush_figures at 0x1061dd710> (for post_execute):


KeyboardInterrupt: 

One can see the class imbalance problem here. Other observations:
  * Heartrate, RRate, ABPm,  distribution is similar to a normal distribution
  * SpO2 is like a censored normal distribution. 
  * For all of the other features, class imbalance is an obvious problem.

A basic strategy that could be used here: Upsample both classes! Do the upsampling efficiently, not just replicating the datapoints

### Train Data pre-processing

In [4]:
# data inspection: 
#############################################
# range of the provided data?
print(df_train_features.agg([min, max]))

# how much missing data? 
print("number of missing values:")
print(df_train_features.isnull().sum(axis=0))

       pid  Time    Age  EtCO2    PTT    BUN  Lactate  Temp   Hgb  HCO3  ...  \
min      1     1   15.0   10.0   12.5    1.0      0.2  21.0   3.3   0.0  ...   
max  31658   315  100.0  100.0  250.0  268.0     31.0  42.0  23.8  50.0  ...   

     Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Heartrate  \
min          12.0   20.0              0.01      66.0   9.4       23.0   
max        3833.0  100.0             21.20     141.0  63.4      191.0   

     Bilirubin_total  TroponinI   ABPs    pH  
min              0.1       0.01   21.0  6.82  
max             46.5     440.00  287.0  7.78  

[2 rows x 37 columns]
number of missing values:
pid                      0
Time                     0
Age                      0
EtCO2               218157
PTT                 217641
BUN                 207835
Lactate             217184
Temp                146825
Hgb                 205645
HCO3                215381
BaseExcess          208053
RRate                40155
Fibrinogen          22544

### Train data pre-processing

In [5]:
# THIS STEP REMOVES ALL NAN !!!

# aggregate data for each pid
# GROUPBY REARRANGES THE ROWS, WE HAVE TO DO THE SAME FOR THE LABELS
df_train_aggregate_features = df_train_features.groupby('pid').agg('sum')

# print(df_train_aggregate_features)

In [6]:
# remove time from data frame 
df_train_agg_features = df_train_aggregate_features.drop(['Time'], axis = 1)
# print(df_train_agg_features)

In [9]:
# impute missing data points
#imp = SimpleImputer(strategy="mean")
imputer = KNNImputer(n_neighbors=12)
df_train_agg_imputed_features = imputer.fit_transform(df_train_agg_features)
#print(df_train_agg_imputed_features)

In [10]:
# scale the data
min_max_scaler = preprocessing.MinMaxScaler()
# standard_scalar = preprocessing.StandardScaler()

data_train_scaled = min_max_scaler.fit_transform(df_train_agg_imputed_features)

In [11]:
# REARRANGE THE LABELS, TO MATCH THE REARRANGED FEATURES
df_train_labels_sorted = df_train_labels.sort_values(by = 'pid')
# print(df_train_labels_sorted)

### Test Data pre-processing

In [12]:
# data inspection: 
#############################################
# range of the provided data?
print(df_test_features.agg([min, max]))

# how much missing data? 
print("number of missing values:")
print(df_test_features.isnull().sum(axis=0))

       pid  Time    Age  EtCO2    PTT    BUN  Lactate  Temp   Hgb  HCO3  ...  \
min      0     1   16.0   10.0   18.1    1.0      0.3  27.0   2.3   0.0  ...   
max  31655   293  100.0  100.0  249.9  205.0     23.3  42.0  23.8  49.0  ...   

     Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Heartrate  \
min          11.0   22.0              0.01      74.0   9.1       21.0   
max        2121.0  100.0             21.00     145.0  71.7      184.0   

     Bilirubin_total  TroponinI   ABPs    pH  
min              0.1       0.01   28.0  6.62  
max             46.4     271.60  281.0  7.69  

[2 rows x 37 columns]
number of missing values:
pid                      0
Time                     0
Age                      0
EtCO2               145440
PTT                 145115
BUN                 138520
Lactate             144692
Temp                 97813
Hgb                 137096
HCO3                143546
BaseExcess          138730
RRate                26669
Fibrinogen          15031

In [13]:
# aggregate data for each pid
df_test_aggregate_features = df_test_features.groupby('pid').agg('sum')

#print(df_test_aggregate_features)

# collect all test pids
test_pids = list(set(df_test_features.pid))

In [14]:
# remove time from data frame 
df_test_agg_features = df_test_aggregate_features.drop(['Time'], axis = 1)
# print(df_test_agg_features)

In [15]:
# impute missing data points
# should we impute it with the same imputer that we've used for train?
df_test_agg_imputed_features = imputer.transform(df_test_agg_features)

In [16]:
# scale test data
data_test_scaled = min_max_scaler.transform(df_test_agg_imputed_features)

## Fit a model & Predict

### predict with support vector machine classification and use probabilities

In [17]:
# first for the labels that have an output [0,1]

columns_1 = [test_pids]

for i in range(1, 12):
    clf = SVC(kernel = 'poly', degree = 5, class_weight = 'balanced', verbose = True)
    clf.fit(data_train_scaled, df_train_labels_sorted.iloc[:,i])
    # pred = clf.predict(df_test_agg_imputed_features)
    # columns_1.append(pred)
     
    # compute probabilites as opposed to predictions
    dual_coefficients = clf.dual_coef_    # do we have to normalize with norm of this vector ?
    distance_hyperplane = clf.decision_function(data_test_scaled)
    probability = np.empty(len(distance_hyperplane))
    for j in range(0, len(probability)):
        probability[j] = 1 / (1 + math.exp(- distance_hyperplane[j]))
    columns_1.append(probability)


    
    distance_hyperplace_train = clf.decision_function(data_train_scaled)
    probability = np.empty(len(distance_hyperplace_train))
    for j in range(0, len(probability)):
        probability[j] = 1 / (1 + math.exp(- distance_hyperplace_train[j]))
      
    tmp = roc_auc_score(y_score= probability, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)
    

[LibSVM]ROC AUC for feature LABEL_BaseExcess  :  0.5075857585724688
[LibSVM]ROC AUC for feature LABEL_Fibrinogen  :  0.507705719969147
[LibSVM]

KeyboardInterrupt: 

In [18]:
# labels that have a real value
columns_2 = []

for i in range(12, 16):
    clf = SVR(kernel = 'rbf', gamma = 'scale')
    clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
    pred_train = clf.predict(data_train_scaled)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
    
    pred = clf.predict(data_test_scaled)
    columns_2.append(pred)

KeyboardInterrupt: 

In [ ]:
columns_final = columns_1 + columns_2


### predict with Support vector regression and then compute sigmoid function

In [ ]:
# first for the labels that have an output [0,1]

columns_1 = [test_pids]

for i in range(1,12):
    
    clf = SVR(kernel = 'poly', degree = 3, max_iter = 10000)
    clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
    pred = clf.predict(data_test_scaled)
    prob = np.empty(len(pred))
    for j in range(0, len(pred)):
        prob[j] = 1 / (1 + math.exp(-pred[j]))
    columns_1.append(prob)
    
    pred_train = clf.predict(data_train_scaled)
    prob_train = np.empty(len(pred_train))
    for j in range(0, len(pred_train)):
        prob_train[j] = 1 / (1 + math.exp(-pred_train[j]))    
    tmp = roc_auc_score(y_score= prob_train, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)


In [21]:
# labels that have a real value

columns_2 = []

for i in range(12, 16):
    clf = LinearSVR()
    clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
    pred = clf.predict(data_test_scaled)
    columns_2.append(pred)
    
    pred_train = clf.predict(data_train_scaled)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 AUC for feature", list(df_train_labels)[i] , " : ", tmp)

R2 AUC for feature LABEL_RRate  :  -0.009527238410467653
R2 AUC for feature LABEL_ABPm  :  -0.018547551642924764
R2 AUC for feature LABEL_SpO2  :  -0.02776805968548679
R2 AUC for feature LABEL_Heartrate  :  -0.002994596868435284


In [ ]:
columns_final = columns_1 + columns_2

## Save predictions

In [ ]:
print(np.shape(columns_final))
result = pd.DataFrame(columns_final).transpose()
result.columns = list(df_train_labels)
result.to_csv('./Results/prediction.csv.zip', index=False, float_format='%.3f', compression='zip')

In [ ]:
result.to_csv('./Results/prediction.csv', index=False, float_format='%.3f')